In [ ]:
import os
os.environ['KAGGLE_USERNAME']="simransohal"
os.environ['KAGGLE_KEY']="3aa7697cd9a4ba2a3715c4a29c0f0cb6"

In [ ]:
!kaggle datasets download -d hhs/health-insurance-marketplace

100% 828M/829M [00:42<00:00, 19.1MB/s]
100% 829M/829M [00:42<00:00, 20.5MB/s]


In [ ]:
ls

health-insurance-marketplace.zip  sample_data/


In [ ]:
import zipfile
with zipfile.ZipFile('health-insurance-marketplace.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
import numpy as np
import pandas as pd
from  matplotlib import pyplot
import seaborn as sns

In [ ]:
ls

BenefitsCostSharing.csv  hashes.txt                        raw/
BusinessRules.csv        health-insurance-marketplace.zip  sample_data/
Crosswalk2015.csv        Network.csv                       ServiceArea.csv
Crosswalk2016.csv        PlanAttributes.csv
database.sqlite          Rate.csv


In [ ]:
rate_fields = ['StateCode','PlanId','IndividualRate','PrimarySubscriberAndThreeOrMoreDependents','BusinessYear','Age','RatingAreaId','Tobacco','SourceName']
rate_chunks = pd.read_csv("Rate.csv",iterator=True,chunksize = 1000, usecols=rate_fields)
rates_2= pd.concat(chunk for chunk in rate_chunks)

In [ ]:
df = rates_2
df['tobacco_rate'] = np.where(df['Tobacco']!= 'Tobacco User/Non-Tobacco User ',df['IndividualRate'],df['IndividualRate']*1.03)
df.head()

,BusinessYear,StateCode,SourceName,PlanId,RatingAreaId,Tobacco,Age,IndividualRate,PrimarySubscriberAndThreeOrMoreDependents,tobacco_rate
0,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,0-20,29.00,NaN,29.00
1,2014,AK,HIOS,21989AK0020001,Rating Area 1,No Preference,Family Option,36.95,107.61,36.95
2,2014,AK,HIOS,21989AK0020001,Rating Area 2,No Preference,Family Option,36.95,107.61,36.95
3,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,21,32.00,NaN,32.00
4,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,22,32.00,NaN,32.00


In [ ]:
df['Age'] = np.where(df['Age']== 'Family Option',-5,df['Age']) # Family Plan = -5
df['Age'] = np.where(df['Age']== '0-20',10,df['Age'])  # Minor = 10 below 20 years of age
df['Age'] = np.where(df['Age']== '65 and over',100,df['Age'])  # Old age = 100 more than 65 years of age

In [ ]:
df['PrimarySubscriberAndThreeOrMoreDependents']=df['PrimarySubscriberAndThreeOrMoreDependents'].fillna(0) 


In [ ]:
df['tobacco_rate'] = np.where(df['PrimarySubscriberAndThreeOrMoreDependents']>0, df['PrimarySubscriberAndThreeOrMoreDependents'],df['tobacco_rate'])
df['PrimarySubscriberAndThreeOrMoreDependents'] = np.where(df['PrimarySubscriberAndThreeOrMoreDependents']>0, 1,df['PrimarySubscriberAndThreeOrMoreDependents'])
df

,BusinessYear,StateCode,SourceName,PlanId,RatingAreaId,Tobacco,Age,IndividualRate,PrimarySubscriberAndThreeOrMoreDependents,tobacco_rate
0,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,10,29.00,0.0,29.00
1,2014,AK,HIOS,21989AK0020001,Rating Area 1,No Preference,-5,36.95,1.0,107.61
2,2014,AK,HIOS,21989AK0020001,Rating Area 2,No Preference,-5,36.95,1.0,107.61
3,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,21,32.00,0.0,32.00
4,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,22,32.00,0.0,32.00
...,...,...,...,...,...,...,...,...,...,...
12694440,2016,WV,SERFF,96480WV0090003,Rating Area 11,No Preference,61,14.05,0.0,14.05
12694441,2016,WV,SERFF,96480WV0090003,Rating Area 11,No Preference,62,14.05,0.0,14.05
12694442,2016,WV,SERFF,96480WV0090003,Rating Area 11,No Preference,63,14.05,0.0,14.05
12694443,2016,WV,SERFF,96480WV0090003,Rating Area 11,No Preference,64,14.05,0.0,14.05


In [ ]:
df.Age = df.Age.astype(int)

In [ ]:
filtered_df = df[df['Tobacco'] == 'Tobacco User/Non-Tobacco User']

# create a copy of the filtered dataframe
new_df = filtered_df.copy()
new_df2 = filtered_df.copy()
# change the value in the Tobacco column to 'Tobacco User' for the first half of the rows
new_df.loc[new_df.index[:len(filtered_df)], 'Tobacco'] = 'Tobacco User'

# change the value in the Tobacco column to 'Non-Tobacco User' for the second half of the rows
new_df2.loc[new_df.index[:len(filtered_df)], 'Tobacco'] = 'Non-Tobacco User'

# replace the tobacco_rate column value with the IndividualRate value for the 'Non-Tobacco User' rows
new_df2.loc[new_df['Tobacco'] == 'Non-Tobacco User', 'tobacco_rate'] = new_df2.loc[new_df['Tobacco'] == 'Non-Tobacco User', 'IndividualRate']

# concatenate the original dataframe with the modified filtered dataframe
ff = pd.concat([new_df,new_df2])
final_df = pd.concat([df[df['Tobacco'] != 'Tobacco User/Non-Tobacco User'], ff])

# reset the index
final_df = final_df.reset_index(drop=True)

final_df

,BusinessYear,StateCode,SourceName,PlanId,RatingAreaId,Tobacco,Age,IndividualRate,PrimarySubscriberAndThreeOrMoreDependents,tobacco_rate
0,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,10,29.00,0.0,29.00
1,2014,AK,HIOS,21989AK0020001,Rating Area 1,No Preference,-5,36.95,1.0,107.61
2,2014,AK,HIOS,21989AK0020001,Rating Area 2,No Preference,-5,36.95,1.0,107.61
3,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,21,32.00,0.0,32.00
4,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,22,32.00,0.0,32.00
...,...,...,...,...,...,...,...,...,...,...
17584562,2016,WV,SERFF,50328WV0020018,Rating Area 11,Non-Tobacco User,61,702.16,0.0,702.16
17584563,2016,WV,SERFF,50328WV0020018,Rating Area 11,Non-Tobacco User,62,717.91,0.0,717.91
17584564,2016,WV,SERFF,50328WV0020018,Rating Area 11,Non-Tobacco User,63,737.65,0.0,737.65
17584565,2016,WV,SERFF,50328WV0020018,Rating Area 11,Non-Tobacco User,64,749.64,0.0,749.64


In [ ]:
final_df.to_csv('clean_2.csv')

In [26]:
import numpy as np
import pandas as pd

# Load the dataset
df = pd.read_csv('clean_2.csv')

sample_size = 70000

# Randomly select data points
sample_indices = np.random.choice(df.index, sample_size, replace=False)

final_df = df.loc[sample_indices]


In [37]:
final_df_tabacoo=df[df["Tobacco"]=="Tobacco User"]
final_df_non_tabacco=df[df["Tobacco"]=="No Preference"]

In [38]:
final_df_tabacoo.head()

,Unnamed: 0,BusinessYear,StateCode,SourceName,PlanId,RatingAreaId,Tobacco,Age,IndividualRate,PrimarySubscriberAndThreeOrMoreDependents,tobacco_rate
7804323,7804323,2014,AL,HIOS,44580AL0360002,Rating Area 3,Tobacco User,10,111.74,0.0,111.74
7804324,7804324,2014,AL,HIOS,44580AL0360002,Rating Area 3,Tobacco User,21,175.99,0.0,175.99
7804325,7804325,2014,AL,HIOS,44580AL0360002,Rating Area 3,Tobacco User,22,175.99,0.0,175.99
7804326,7804326,2014,AL,HIOS,44580AL0360002,Rating Area 3,Tobacco User,23,175.99,0.0,175.99
7804327,7804327,2014,AL,HIOS,44580AL0360002,Rating Area 3,Tobacco User,24,175.99,0.0,175.99


In [40]:
final_df_non_tabacco.head()

,Unnamed: 0,BusinessYear,StateCode,SourceName,PlanId,RatingAreaId,Tobacco,Age,IndividualRate,PrimarySubscriberAndThreeOrMoreDependents,tobacco_rate
0,0,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,10,29.00,0.0,29.00
1,1,2014,AK,HIOS,21989AK0020001,Rating Area 1,No Preference,-5,36.95,1.0,107.61
2,2,2014,AK,HIOS,21989AK0020001,Rating Area 2,No Preference,-5,36.95,1.0,107.61
3,3,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,21,32.00,0.0,32.00
4,4,2014,AK,HIOS,21989AK0010001,Rating Area 1,No Preference,22,32.00,0.0,32.00


In [28]:
features = ['BusinessYear', 'StateCode', 'SourceName', 'RatingAreaId', 'Tobacco', 'Age', 'PrimarySubscriberAndThreeOrMoreDependents','tobacco_rate']
target = 'PlanId'
final_df.head()

,Unnamed: 0,BusinessYear,StateCode,SourceName,PlanId,RatingAreaId,Tobacco,Age,IndividualRate,PrimarySubscriberAndThreeOrMoreDependents,tobacco_rate
3542766,3542766,2015,TN,HIOS,14002TN0190100,Rating Area 8,No Preference,56,1009.40,0.0,1009.40
1052782,1052782,2014,SC,HIOS,77305SC0170001,Rating Area 42,No Preference,61,27.86,0.0,27.86
10724968,10724968,2015,IL,SERFF,20129IL0280014,Rating Area 13,Tobacco User,23,405.08,0.0,405.08
7585410,7585410,2016,OH,SERFF,86728OH0290003,Rating Area 8,No Preference,61,24.30,0.0,24.30
5133466,5133466,2016,AL,HIOS,60075AL0020001,Rating Area 10,No Preference,60,29.86,0.0,29.86


In [41]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn import tree
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.ensemble import BaggingClassifier
le = LabelEncoder()
final_df_tabacoo['SourceName'] = le.fit_transform(final_df_tabacoo['SourceName'])
final_df_tabacoo['StateCode'] = le.fit_transform(final_df_tabacoo['StateCode'])
final_df_tabacoo['Tobacco'] = le.fit_transform(final_df_tabacoo['Tobacco'])
final_df_tabacoo['PlanId'] = le.fit_transform(final_df_tabacoo['PlanId'])
final_df_tabacoo['RatingAreaId'] = le.fit_transform(final_df_tabacoo['RatingAreaId'])

final_df_non_tabacco['SourceName'] = le.fit_transform(final_df_non_tabacco['SourceName'])
final_df_non_tabacco['StateCode'] = le.fit_transform(final_df_non_tabacco['StateCode'])
final_df_non_tabacco['Tobacco'] = le.fit_transform(final_df_non_tabacco['Tobacco'])
final_df_non_tabacco['PlanId'] = le.fit_transform(final_df_non_tabacco['PlanId'])
final_df_non_tabacco['RatingAreaId'] = le.fit_transform(final_df_non_tabacco['RatingAreaId'])

<ipython-input-41-ba3e5fe4f16a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_tabacoo['SourceName'] = le.fit_transform(final_df_tabacoo['SourceName'])
<ipython-input-41-ba3e5fe4f16a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_tabacoo['StateCode'] = le.fit_transform(final_df_tabacoo['StateCode'])
<ipython-input-41-ba3e5fe4f16a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [45]:
import numpy as np
num_rows = final_df_non_tabacco.shape[0]
random_rows = np.random.choice(num_rows, size=int(num_rows*0.2), replace=False)
subset_df = final_df_non_tabacco.iloc[random_rows, :]
subset_df['SourceName'] = le.fit_transform(subset_df['SourceName'])
subset_df['StateCode'] = le.fit_transform(subset_df['StateCode'])
subset_df['Tobacco'] = le.fit_transform(subset_df['Tobacco'])
subset_df['PlanId'] = le.fit_transform(subset_df['PlanId'])
subset_df['RatingAreaId'] = le.fit_transform(subset_df['RatingAreaId'])

<ipython-input-45-ec9be3fa791d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['SourceName'] = le.fit_transform(subset_df['SourceName'])
<ipython-input-45-ec9be3fa791d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['StateCode'] = le.fit_transform(subset_df['StateCode'])
<ipython-input-45-ec9be3fa791d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [46]:
import numpy as np
num_rows = final_df_tabacoo.shape[0]
random_rows = np.random.choice(num_rows, size=int(num_rows*0.2), replace=False)
subset_df_t = final_df_tabacoo.iloc[random_rows, :]
subset_df_t['SourceName'] = le.fit_transform(subset_df_t['SourceName'])
subset_df_t['StateCode'] = le.fit_transform(subset_df_t['StateCode'])
subset_df_t['Tobacco'] = le.fit_transform(subset_df_t['Tobacco'])
subset_df_t['PlanId'] = le.fit_transform(subset_df_t['PlanId'])
subset_df_t['RatingAreaId'] = le.fit_transform(subset_df_t['RatingAreaId'])

<ipython-input-46-b8bc58c73fd8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df_t['SourceName'] = le.fit_transform(subset_df_t['SourceName'])
<ipython-input-46-b8bc58c73fd8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df_t['StateCode'] = le.fit_transform(subset_df_t['StateCode'])
<ipython-input-46-b8bc58c73fd8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [55]:
model=tree.DecisionTreeClassifier()

In [56]:
#bag_clf = BaggingClassifier(base_estimator=model, n_estimators=10, max_samples=0.01, max_features=5)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(subset_df_t[['BusinessYear', 'StateCode', 'SourceName', 'Tobacco', 'RatingAreaId','Age', 'PrimarySubscriberAndThreeOrMoreDependents']],subset_df_t[target],test_size=0.3)
model.fit(X_train, y_train)                                                      

DecisionTreeClassifier()

In [50]:
plt.figure(figsize=(300,250))
plot_tree(model, filled=True)
plt.show()

KeyboardInterrupt: ignored

In [ ]:
y_pred=model.predict(X_test)

print(sklearn.metrics.f1_score(y_test,y_pred,average='weighted'))

In [25]:
X=subset_df[features]
y=subset_df[target]


In [ ]:
X

['BusinessYear',
 'StateCode',
 'SourceName',
 'RatingAreaId',
 'Tobacco',
 'Age',
 'PrimarySubscriberAndThreeOrMoreDependents',
 'tobacco_rate']

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split

# load your data
X=subset_df[features]
y=subset_df[target]

# split the data into chunks
chunk_size = 1000
X_chunks = [X[i:i+chunk_size] for i in range(0, len(X), chunk_size)]
y_chunks = [y[i:i+chunk_size] for i in range(0, len(y), chunk_size)]

# train a decision tree on each chunk
tree_models = []
for X_chunk, y_chunk in zip(X_chunks, y_chunks):
    model = DecisionTreeClassifier(max_depth=15)
    model.fit(X_chunk, y_chunk)
    tree_models.append( model)

# ensemble the models into a single model



In [ ]:
tree_models_1 = []
x = 0
for i in tree_models:
  x+=1
  tree_models_1.append(('tree'+str(x),i))


In [ ]:
ensemble_model = VotingClassifier(tree_models_1)

# train the ensemble model on all the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
ensemble_model.fit(X_train, y_train)

# evaluate the ensemble model
ensemble_score = ensemble_model.score(X_test, y_test)

print("Ensemble model score:", ensemble_score)